## SVM 만들어보기

- 어떤 수식이 필요할까? - 
https://ratsgo.github.io/machine%20learning/2017/05/23/SVM/

- 마진을 최대화 시키는 경계를 찾는 목적식, 이 목적식에 대한 제약식
- '제약 조건이 있는 최적화 문제를 풀기' -> '최적점이 되기위한 조건식을 푸는 문제'로 바꿔주는 라그랑주 승수법 사용
- 라그랑주 함수에서 나오는 알파는 KKT(Karush-Kuhn-Tucker) 승수
- KKT 조건을 만족하는 정류점(stationary point)에 해가 있음(존재한다면)
- 라그랑주 함수가 KTT 조건을 만족하는지 보이고(결과는 당연히 필요충분을 만족함)
- 정류점의 속성을 대입해서 SVM문제를 쌍대 형식으로 나타내고 그 함수를 최소화하는 (+ 모든 알파추정치에서 양수인) 알파 추정치를 찾는 문제( = 쌍대 문제 )
- 쌍대 문제에서 구한 w와 b의 추정치로 원래 마진에 업데이트<br/><br/>

- kernel의 핵심 : (2차 다항식)변환된 벡터의 스칼라 곱이 원래 벡터 스칼라 곱의 제곱과 같음